# NWB workshop
## Converting to NWB with neuroconv
We will deal with two problems that we think arise often.
The first one is having a type of data that you would like
to convert to an NWB file and the second you already have an
NWB and some data you would like to add to the NWB file.

## Prerequisites
If you were running this on your own machine you would have
to install jupyter, neuroconv package and the data.
However for the workshop these steps are baked in
the provided notebooks so you don't need to anything.

## Problem 1:
We will start with the simplest case scenario. The data that we have
is from a csv file. 

In [1]:
from neuroconv.datainterfaces import CsvTimeIntervalsInterface
from zoneinfo import ZoneInfo
from pathlib import Path
from datetime import datetime

/home/apitsill/ephemera/nwb_workshops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import pandas as pd

In [3]:
n = 100
starts  = [-1]*n
ends  = [-1]*n
vals = [-1]*n
s0 = 0
for i in range(n):
    starts[i]= s0 + random.randrange(5)
    ends[i] = starts[i] + random.randrange(1,50)
    vals[i] = random.uniform(0,1)
    s0 = ends[i]
    

pd.DataFrame({'start_time':starts, 'end_time':ends, 'value':vals }).to_csv("mydata.csv",index=False)

In [4]:
csv_file_path = "mydata.csv"
nwb_file_path = "csv_file.nwb"
interface = CsvTimeIntervalsInterface(file_path=csv_file_path, verbose=False)

In [5]:
metadata = interface.get_metadata()
session_start_time = datetime(2025,1,10,11,45,0, tzinfo=ZoneInfo("Europe/Paris"))
metadata

DeepDict: {'NWBFile': {'session_description': '', 'identifier': '7929d8c8-25ed-4df5-84e7-3d3d60216283', 'source_script': 'Created using NeuroConv v0.6.6', 'source_script_file_name': '/home/apitsill/ephemera/nwb_workshops/lib/python3.11/site-packages/neuroconv/basedatainterface.py'}, 'TimeIntervals': {'trials': {'table_name': 'trials', 'table_description': 'experimental trials generated from mydata.csv'}}}

In [6]:
interface.run_conversion(nwbfile_path = nwb_file_path, metadata=metadata)

ValidationError: 'session_start_time' is a required property

Failed validating 'required' in schema['properties']['NWBFile']:
    {'type': 'object',
     'additionalProperties': False,
     'required': ['session_start_time'],
     'properties': {'keywords': {'description': 'Terms to search over',
                                 'type': 'array',
                                 'items': {'title': 'keyword',
                                           'type': 'string'}},
                    'experiment_description': {'type': 'string',
                                               'description': 'general '
                                                              'description '
                                                              'of the '
                                                              'experiment'},
                    'session_id': {'type': 'string',
                                   'description': 'lab-specific ID for the '
                               

In [7]:
metadata["NWBFile"]["session_start_time"] = session_start_time

In [8]:
interface.run_conversion(nwbfile_path = nwb_file_path, metadata=metadata)

In [9]:
from pynwb import NWBHDF5IO

In [10]:
with NWBHDF5IO(nwb_file_path, mode = 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    starts = nwbfile.trials['start_time'].data
    print(starts[0:5])

root pynwb.file.NWBFile at 0x139990873764048
Fields:
  file_create_date: [datetime.datetime(2025, 1, 10, 12, 5, 10, 256411, tzinfo=tzoffset(None, 3600))]
  identifier: 7929d8c8-25ed-4df5-84e7-3d3d60216283
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  session_start_time: 2025-01-10 11:45:00+01:00
  source_script: Created using NeuroConv v0.6.6
  source_script_file_name: /home/apitsill/ephemera/nwb_workshops/lib/python3.11/site-packages/neuroconv/basedatainterface.py
  timestamps_reference_time: 2025-01-10 11:45:00+01:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>

[ 3. 25. 74. 81. 97.]
